In [ ]:
import sys
import os
import time 
# Add project root to Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS 
from src.utils import get_config

# Get the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

config = get_config(config_path=os.path.join(PROJECT_ROOT, "config.yaml")) 

embeddings = OpenAIEmbeddings(
    api_key=config['openai']['token'],
    model="text-embedding-3-small"
    ) 

# --- ส่วนของการตั้งค่า Path ---
DATA_FOLDER = os.path.join(PROJECT_ROOT, "data/raw")  # ใช้ path แบบสัมบูรณ์
FILE_NAMES = ["workshop.txt", "rerun.txt", "overall.txt"]
FAISS_INDEX_PATH = os.path.join(PROJECT_ROOT, "artifacts/faiss_product_index")  # ใช้ path แบบสัมบูรณ์

# สร้าง DATA_PATHS จาก DATA_FOLDER และ FILE_NAMES
DATA_PATHS = [os.path.join(DATA_FOLDER, fname) for fname in FILE_NAMES]

# --- ส่วนของการโหลด/สร้าง Index ---
index_file = os.path.join(FAISS_INDEX_PATH, "index.faiss")

if os.path.exists(index_file):
    print(f"\n✅ พบ Index ที่บันทึกไว้, กำลังโหลดจาก '{FAISS_INDEX_PATH}'...") 
    vectorstore = FAISS.load_local(
        FAISS_INDEX_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )
    print(f"✅ โหลด Index สำเร็จ")
else:
    print(f"\n🟡 ไม่พบ Index, กำลังสร้าง Index ใหม่จากไฟล์: {', '.join(FILE_NAMES)}")
    all_docs = []
    for path in DATA_PATHS:
        if not os.path.exists(path):
            print(f"‼️ ข้อผิดพลาด: ไม่พบไฟล์ที่ '{path}' กรุณาตรวจสอบตำแหน่งไฟล์")
            continue
        print(f"  - กำลังโหลดไฟล์: {path}")
        loader = TextLoader(path, encoding="utf-8")
        docs_from_file = loader.load()
        all_docs.extend(docs_from_file)

    if not all_docs:
        print("\n‼️ ข้อผิดพลาดร้ายแรง: ไม่สามารถโหลดเอกสารใดๆ ได้เลย")
        print("กรุณาตรวจสอบ Path และโครงสร้างโฟลเดอร์ให้ถูกต้อง")
    else:
        print(f"\n✅ โหลดเอกสารทั้งหมด {len(all_docs)} document(s) เรียบร้อย")

        text_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile"
        )
        splits = text_splitter.split_documents(all_docs)
        print(f"✅ แบ่งเอกสารออกเป็น {len(splits)} chunks")

        print("⏳ กำลังสร้าง Vector Store ด้วย FAISS (อาจใช้เวลาสักครู่)...")
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(FAISS_INDEX_PATH)
        print(f"💾 Index ถูกสร้างและบันทึกไว้ที่ '{FAISS_INDEX_PATH}'")


✅ พบ Index ที่บันทึกไว้, กำลังโหลดจาก '/home/godoftan/dev/typhoon/artifacts/faiss_product_index'...
✅ โหลด Index สำเร็จใน 0.03 วินาที


In [10]:
# สร้าง retriever จาก vectorstore (เพิ่มส่วนนี้เพื่อแก้ไข NameError)
res = vectorstore.similarity_search(
    query="มีอะไรบ้าง?",
    search_type="similarity",
    search_kwargs={"k": 4}  # จำนวน chunks ที่จะดึงมา
) 

In [12]:
print(res[0].page_content)

- **ตอบ:** เพื่อให้ข้อมูลเชิงลึกเกี่ยวกับข้อมูล Quant และกลยุทธ์การเทรด

**Q&A Section**

Q: ผมยังไม่รู้จะไปทางไหน? A: คอร์สนี้จะทำให้คุณเห็นภาพกว้างมากขึ้น

Q: อยากทำระบบเทรด Robot Trade แต่ไม่มี Idea
A: คุณจะได้ Idea มากมายที่ Speaker แต่ละท่านใช้

Q: อยากทำงานด้าน Quant Trade การลงทุน
A: บอกเลยครับว่าทำได้ เพราะ speaker แต่ละท่านนี่แทบจะไม่ได้เรียนมาโดยตรงด้านนี้เลยครับ

**Workshops Overview**

- Online Workshops "จับมือเทรดด้วย Quant Data"
- สิ่งที่คุณจะได้รับเพียงซื้อ 1 Workshop ขึ้นไป ในราคา 3,599 บาท
  - RERUN ทุก Session ทั้ง Stage 1 & 2 (รวม 11 ชั่วโมง)
  - RERUN Crypto Quant Trading (รวม 7 ชั่วโมง)
  - สิทธิ์ใช้เครื่องมือดู Quant Tool ของ Investic Analytics Studio เป็นระยะเวลา 1 เดือน

**Rerun Quant Offside 3x มีเนื้อหาอะไรบ้าง**

- การใช้ Market Breadth ในการ Filter ตลาด
- ตรวจดูพฤติกรรมหุ้นที่แข็งกว่าตลาด
- สถิติตลาดหุ้นในอดีตมีลักษณะรูปแบบไหน
- Use case การพัฒนาระบบเทรดในตลาดไทย
- การประเมินมูลค่าหุ้นด้วย Fundamental Analysis
- การนำ AI มาช่วยทำให้ Fundamental Analysis ให้